In [33]:
import pandas as pd
import json

In [34]:
epi_r_df = pd.read_csv('epi_r.csv')
epi_r_df.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
with open('epi_r.columns.json', 'w', encoding='UTF-8') as wp:
    a = epi_r_df.columns.tolist()
    json.dump(a, wp)

In [36]:
# 只需要
# 0: "标题" "评分" "卡路里" "蛋白质" "脂肪" "钠含量"
epi_r_df_selected = epi_r_df[['title', 'rating', 'calories', 'protein', 'fat', 'sodium']]
epi_r_df_selected = epi_r_df_selected.drop_duplicates()
epi_r_df_selected.shape

(18235, 6)

In [37]:
with open('full_format_recipes.json', 'r', encoding='UTF-8') as fp:
    recipe_data = json.load(fp)

In [38]:
# 需要category
recipe_df_data = []
for row in recipe_data:
    if 'title' in row and 'categories' in row:
        categories = ';'.join(row['categories'])
        recipe_df_data.append([row['title'], categories])
recipe_df = pd.DataFrame(data=recipe_df_data, columns=['title', 'categories'])
recipe_df = recipe_df.drop_duplicates('title')

In [39]:
df = pd.merge(epi_r_df_selected, recipe_df, how='left', on='title')
df.head()

,title,rating,calories,protein,fat,sodium,categories
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,Sandwich;Bean;Fruit;Tomato;turkey;Vegetable;Ki...
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,Food Processor;Onion;Pork;Bake;Bastille Day;Ne...
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,Soup/Stew;Dairy;Potato;Vegetable;Fennel;Gourme...
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,Fish;Olive;Tomato;Sauté;Low Fat;Low Cal;High F...
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,Cheese;Dairy;Pasta;Vegetable;Side;Bake;Vegetar...


In [40]:
df.columns

Index(['title', 'rating', 'calories', 'protein', 'fat', 'sodium',
       'categories'],
      dtype='object')

In [41]:
df.to_csv('data_processed.csv', index=False, encoding='UTF-8')

In [42]:
# 导入到sqllite

import sqlalchemy

# 定义SQLite数据库文件的路径
database_path = '../mysite/db.sqlite3'

# 创建数据库引擎，SQLite URI通常以'sqlite:///...'形式给出
# 注意前三个斜线，第一个双斜线意味着使用本地文件系统
engine = sqlalchemy.create_engine('sqlite:///{}'.format(database_path))

sql = 'delete from app_data'
conn = engine.connect()
conn.execute(sql)
conn.close()

df.head(500).to_sql('app_data', con=engine, if_exists='append', index=False)